In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display


class ExcelTableReader:
    def __init__(self, url):
        self.url = url
        self.data = None

    def load_excel(self):
        try:
            self.data = pd.read_excel(self.url, sheet_name=None)
            print("Excel file loaded successfully.")
        except Exception as e:
            print(f"Error loading Excel file: {str(e)}")

    def get_sheet_names(self):
        if self.data is not None:
            return list(self.data.keys())
        else:
            print("No Excel file loaded.")
            return []

    def get_table_names(self, sheet_name):
        if sheet_name in self.data.keys():
            sheet_data = self.data[sheet_name]
            table1_name = sheet_data.iloc[3, 6]
            table2_name = sheet_data.iloc[4, 0]
            return table1_name, table2_name
        else:
            print(f"Sheet '{sheet_name}' does not exist.")
            
    def get_table(self, sheet_name, table_name):
        if sheet_name in self.data.keys():
            sheet_data = self.data[sheet_name]
            if table_name == self.get_table_names(sheet_name)[0]:
                return sheet_data.iloc[5:16, 6:13+1]
            elif table_name == self.get_table_names(sheet_name)[1]:
                return sheet_data.iloc[5:16, 0:4+1]
            else:
                print("Invalid table name.")
        else:
            print(f"Sheet '{sheet_name}' does not exist.")
        
    def create_dropdown(self):
        sheet_names = self.get_sheet_names()
        sheet_dropdown = widgets.Dropdown(
            options = sheet_names,
            description = "Select sheet:",
        )
        table_dropdown = widgets.Dropdown(
            description="Select table:",
        )
        toggle_button = widgets.ToggleButtons(
            options = ["View", "Save"],
            button_style = "primary",
            description = "Action:",
        )
        button = widgets.Button(description="Execute")
        output = widgets.Output()
        
        def update_table_dropdown(change):
            selected_sheet = sheet_dropdown.value
            table_names = self.get_table_names(selected_sheet)
            table_dropdown.options = table_names

        def execute_action(btn):
            selected_sheet = sheet_dropdown.value
            selected_table = table_dropdown.value
            action = toggle_button.value

            if action == "View":
                table = self.get_table(selected_sheet, selected_table)
                if table is not None:
                    with output:
                        output.clear_output()
                        print(table)
            elif action == "Save":
                table = self.get_table(selected_sheet, selected_table)
                if table is not None:
                    table.to_excel(f"{selected_sheet}_{selected_table}.xlsx")
                    print(f"Table saved as {selected_sheet}_{selected_table}.xlsx")
        
        sheet_dropdown.observe(update_table_dropdown, "value")
        button.on_click(execute_action)
        display(widgets.VBox([sheet_dropdown, table_dropdown, toggle_button, button, output]))


if __name__ == "__main__":
    url = "https://www.nrel.gov/wind/nwtc/assets/downloads/MetOcean/DistributionParameters.xlsx"
    reader = ExcelTableReader(url)
    reader.load_excel()
    reader.create_dropdown()


Excel file loaded successfully.


Table saved as West Coast_Expected Values.xlsx
